In [13]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import sklearn.preprocessing

# --- 1. Lorenz System Definition (aus dem Originalcode) ---
rho = 45.92
sigma = 16.0
beta = 8.0 / 2.0

def lorenz_equation(state, t):
    x, y, z = state
    return sigma * (y - x), x * (rho - z) - y, x * y - beta * z

def lorenz(t, init):
    return odeint(lorenz_equation, init, t)

# --- 2. Datengenerierung (aus dem Originalcode) ---
def generate_data(t, samples=1, mixtures=[], inits=[], noise=0.6):
    assert(len(mixtures) == len(inits))
    candidates = []
    data = np.zeros((samples, t.shape[0], 1))
    candidate_index = np.zeros((samples), dtype=int)

    for init in inits:
        candidates.append(lorenz(t, init)[:, 0:1].tolist())
    candidates = np.array(candidates)

    randoms = np.random.uniform(size=samples)
    for i in range(len(mixtures)):
        if i == 0:
            indices = np.where((randoms >= 0) & (randoms < mixtures[i]))[0]
        else:
            indices = np.where((randoms >= mixtures[i-1]) & (randoms < mixtures[i]))[0]
        for index in indices:
            data[index] = candidates[i] + np.random.normal(loc=0.0, scale=noise, size=candidates[i].shape)
            candidate_index[index] = i
    return data, candidate_index, t

# --- 3. Rebinning Funktion (aus dem Originalcode) ---
def rebin(arr, new_shape):
    shape = (new_shape[0], arr.shape[0] // new_shape[0],
             new_shape[1], arr.shape[1] // new_shape[1],)
    return arr.reshape(shape).mean(-1).mean(1)

# --- 4. Parameter und Generierung der Rohdaten (entspricht In [6] - In [10] im Originalcode) ---

t_full = np.arange(0.0, 26.0, 0.02) # Originale Zeitachse

inits = [
    [ 1.0, 1.0001         , 1.0 ],
    [ 1.0, 1.000001       , 1.0 ],
    [ 1.0, 1.00000001     , 1.0 ],
    [ 1.0, 1.0000000001   , 1.0 ],
    [ 1.0, 1.000000000001 , 1.0 ],
]

# Generiere Mischungsverhältnisse
num_candidates = 5
points = 1000 # Dies war die Basis für die Mischungsverhältnisse, nicht die Samples
distr = np.clip(np.random.normal(loc=0.5, scale=0.2, size=points), a_min=0, a_max=1)
mixtures, edges = np.histogram(distr, bins=num_candidates)
mixtures = mixtures / points
tmp = 0
for i, mixture in enumerate(mixtures):
    tmp += mixture
    mixtures[i] = tmp

# Daten generieren
num_samples = 1000 # Für dieses Beispiel reduzieren wir die Anzahl der Samples, um es übersichtlicher zu halten
                   # Im Originalcode waren es 100000, das würde sehr groß werden!
data_raw, indices, t_generated = generate_data(t_full, samples=num_samples, mixtures=mixtures, inits=inits, noise=7.2)

# Rebinning und Skalierung
scaler = sklearn.preprocessing.MinMaxScaler(feature_range=(-10, 10))
rebinned_data = rebin(data_raw[:, :, 0], (data_raw.shape[0], data_raw.shape[1] // 10))
rebinned_time = t_full[::10] # Die neue Zeitachse nach dem Rebinning

scaler.fit(rebinned_data)
rebinned_data_scaled = scaler.transform(rebinned_data)
# rebinned_data_scaled hat nun Shape (num_samples, num_rebinned_timepoints)
# z.B. (1000, 130)

print(f"Shape der skalierten, rebinned Daten: {rebinned_data_scaled.shape}")
print(f"Anzahl der Zeitpunkte pro Serie (nach Rebinning): {rebinned_data_scaled.shape[1]}")

# --- 5. Transformation ins Long-Format ---

long_format_data = []
# Generische Start-Unix-Zeit in Nanosekunden (z.B. 1. Januar 2020)
start_unix_ns = pd.Timestamp('2020-01-01 00:00:00 UTC').value

# Zeitintervall in Nanosekunden (entspricht dem Zeitabstand in rebinned_time)
# rebinned_time.shape[1] = 130
# t_full.shape[0] = 1300
# t_full[0] = 0.0, t_full[1] = 0.02, ... t_full[1299] = 25.98
# rebinned_time[0] = 0.0, rebinned_time[1] = 0.2, ... rebinned_time[129] = 25.8
# Der Zeitabstand ist 0.2 Sekunden (t_full[::10])
# 0.2 Sekunden in Nanosekunden: 0.2 * 1,000,000,000 = 200,000,000 ns
time_step_ns = (rebinned_time[1] - rebinned_time[0]) * 1_000_000_000 


for i in range(num_samples):
    # Generiere Unix-Zeiten für diese Serie
    # Jedes Sample hat die gleichen Zeitstempel relativ zum Start
    current_series_times = np.array([start_unix_ns + j * time_step_ns for j in range(rebinned_data_scaled.shape[1])])
    
    # Extrahiere die Daten für das aktuelle Sample
    series_data = rebinned_data_scaled[i] # Dies ist ein 1D-Array von 130 Werten

    # Erstelle einen temporären DataFrame für diese eine Zeitreihe
    df_temp = pd.DataFrame({
        'date': current_series_times,
        'data': series_data,
        'cols': f'Channel_{i}' # Jedes Sample wird ein eigener "Kanal"
    })
    long_format_data.append(df_temp)

# Verkette alle temporären DataFrames
df_time_series_long = pd.concat(long_format_data, ignore_index=True)

print("\n✨ Lorenz-Zeitreihen im Long-Format erstellt!")
print("\n📦 Erste 5 Zeilen des Long-Format DataFrames:")
print(df_time_series_long.head())

print("\n📊 DataFrame Info:")
df_time_series_long.info()

print(f"\n📏 Gesamt-DataFrame Shape: {df_time_series_long.shape}")

# Optional: Speichern als CSV
# df_time_series_long.to_csv("lorenz_time_series_long_format.csv", index=False)
# print("\nDatei 'lorenz_time_series_long_format.csv' wurde gespeichert.")

Shape der skalierten, rebinned Daten: (1000, 130)
Anzahl der Zeitpunkte pro Serie (nach Rebinning): 130

✨ Lorenz-Zeitreihen im Long-Format erstellt!

📦 Erste 5 Zeilen des Long-Format DataFrames:
           date      data       cols
0  1.577837e+18  5.362418  Channel_0
1  1.577837e+18  1.933693  Channel_0
2  1.577837e+18  2.452592  Channel_0
3  1.577837e+18  3.211117  Channel_0
4  1.577837e+18  3.911674  Channel_0

📊 DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130000 entries, 0 to 129999
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   date    130000 non-null  float64
 1   data    130000 non-null  float64
 2   cols    130000 non-null  object 
dtypes: float64(2), object(1)
memory usage: 3.0+ MB

📏 Gesamt-DataFrame Shape: (130000, 3)


In [14]:
df_time_series_long

,date,data,cols
0,1.577837e+18,5.362418,Channel_0
1,1.577837e+18,1.933693,Channel_0
2,1.577837e+18,2.452592,Channel_0
3,1.577837e+18,3.211117,Channel_0
4,1.577837e+18,3.911674,Channel_0
...,...,...,...
129995,1.577837e+18,6.690909,Channel_999
129996,1.577837e+18,7.057986,Channel_999
129997,1.577837e+18,5.313935,Channel_999
129998,1.577837e+18,-4.841353,Channel_999


In [15]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import sklearn.preprocessing
from numba import jit # Importiere Numba für Optimierungen
import time # Für Performance-Messung

# --- 1. Lorenz System Definition (mit Numba-Optimierung) ---
rho = 45.92
sigma = 16.0
beta = 8.0 / 2.0

@jit(nopython=True)
def lorenz_equation(state, t):
    """Die Lorenz-Gleichungen, optimiert mit Numba."""
    x, y, z = state
    return sigma * (y - x), x * (rho - z) - y, x * y - beta * z

def lorenz(t, init):
    """Löst das Lorenz-System für gegebene Startwerte und Zeitpunkte."""
    # odeint ruft die Numba-optimierte lorenz_equation auf
    return odeint(lorenz_equation, init, t)

# --- 2. Numba-optimierte Helferfunktion für Mischen und Rauschen ---
@jit(nopython=True)
def _mix_and_noise(data_out, candidates_arr, randoms, mixtures, noise):
    """Numba-optimierte Logik zum Mischen von Kandidaten und Hinzufügen von Rauschen."""
    num_samples = data_out.shape[0]
    num_mixtures = len(mixtures)
    
    for idx in range(num_samples):
        current_random = randoms[idx]
        selected_candidate_idx = -1
        
        for mix_idx in range(num_mixtures):
            if mix_idx == 0:
                if current_random >= 0 and current_random < mixtures[mix_idx]:
                    selected_candidate_idx = mix_idx
                    break
            else:
                if current_random >= mixtures[mix_idx-1] and current_random < mixtures[mix_idx]:
                    selected_candidate_idx = mix_idx
                    break
        
        if selected_candidate_idx != -1:
            noise_array = np.random.normal(loc=0.0, scale=noise, size=data_out[idx].shape)
            data_out[idx] = candidates_arr[selected_candidate_idx] + noise_array
    return data_out

# --- 3. Haupt-Daten-Generierungsfunktion (ruft Numba-Teil auf) ---
def generate_data_optimized(t_full, samples, mixtures, inits, noise):
    """Generiert die Lorenz-Zeitreihendaten, indem sie Numba-optimierte Teile nutzt."""
    print(f"Generiere {len(inits)} Lorenz-Kandidaten (Länge: {t_full.shape[0]} Punkte)...")
    candidates_list = []
    start_candidates = time.time()
    for init_idx, init in enumerate(inits):
        print(f"  Simuliere Kandidat {init_idx + 1}/{len(inits)}...")
        candidates_list.append(lorenz(t_full, init)[:, 0:1])
    candidates_arr = np.array(candidates_list)
    print(f"  Kandidaten-Simulation beendet in {time.time() - start_candidates:.2f} Sekunden.")

    print(f"Mische und füge Rauschen hinzu für {samples} Samples (Numba-optimiert)...")
    data_out = np.zeros((samples, t_full.shape[0], 1), dtype=np.float64)
    randoms = np.random.uniform(size=samples)

    start_mix_noise = time.time()
    data_out = _mix_and_noise(data_out, candidates_arr, randoms, mixtures, noise)
    print(f"  Mischen und Rauschen beendet in {time.time() - start_mix_noise:.2f} Sekunden.")
    
    return data_out, None, t_full

# --- 4. Numba-optimierte Rebinning-Funktion ---
@jit(nopython=True)
def rebin(arr, new_shape):
    """Numba-optimierte Funktion zum Rebinning eines 3D-Arrays."""
    shape = (new_shape[0], arr.shape[0] // new_shape[0],
             new_shape[1], arr.shape[1] // new_shape[1],)
    
    reshaped_arr = arr.reshape(shape)
    
    mean_last_axis = np.mean(reshaped_arr, axis=3)
    mean_second_axis = np.mean(mean_last_axis, axis=1)
    
    return mean_second_axis

# --- 5. Hauptskript: Parameter und Datengenerierung ---
print("🚀 Starte Generierung der Lorenz-Zeitreihen mit Numba-Optimierung...")

# Anpassung: Sehr lange Zeitachse für 10.000 Einträge pro Kanal
# (10.000 Punkte * Rebinning-Faktor 100 = 1.000.000 Rohpunkte)
t_full = np.arange(0.0, 20000.0, 0.02) # Erzeugt 1.000.000 Rohpunkte
num_samples = 10 # Anzahl der Kanäle

# Initialparameter für Lorenz-Simulationen (unverändert)
inits = [
    [ 1.0, 1.0001         , 1.0 ],
    [ 1.0, 1.000001       , 1.0 ],
    [ 1.0, 1.00000001     , 1.0 ],
    [ 1.0, 1.0000000001   , 1.0 ],
    [ 1.0, 1.000000000001 , 1.0 ],
]

# Mischungsverhältnisse generieren (unverändert)
num_candidates = 5
points = 1000 
distr = np.clip(np.random.normal(loc=0.5, scale=0.2, size=points), a_min=0, a_max=1)
mixtures, edges = np.histogram(distr, bins=num_candidates)
mixtures = mixtures / points
tmp = 0
for i, mixture in enumerate(mixtures):
    tmp += mixture
    mixtures[i] = tmp
mixtures_arr = np.array(mixtures, dtype=np.float64) # Für Numba als NumPy-Array

# Daten generieren mit der optimierten Funktion
start_total_generation = time.time()
data_raw, _, t_generated = generate_data_optimized(t_full, samples=num_samples, mixtures=mixtures_arr, inits=inits, noise=7.2)
print(f"Gesamte Rohdaten-Generierung beendet in {time.time() - start_total_generation:.2f} Sekunden.")

# Rebinning-Faktor anpassen für 10.000 Punkte
REBIN_FACTOR = 100 
print(f"\nRebinning der Daten mit Faktor {REBIN_FACTOR} (Numba-optimiert)...")
start_rebin = time.time()
rebinned_data = rebin(data_raw[:, :, 0], (data_raw.shape[0], data_raw.shape[1] // REBIN_FACTOR))
rebinned_time = t_full[::REBIN_FACTOR] # Die Zeitpunkte nach dem Rebinning
print(f"Rebinning beendet in {time.time() - start_rebin:.2f} Sekunden.")

# Skalierung der Daten
print("Skaliere die Daten...")
scaler = sklearn.preprocessing.MinMaxScaler(feature_range=(-10, 10))
scaler.fit(rebinned_data)
rebinned_data_scaled = scaler.transform(rebinned_data)

print(f"Shape der skalierten, rebinned Daten: {rebinned_data_scaled.shape} (Soll: {num_samples}x{t_full.shape[0] // REBIN_FACTOR})")
print(f"Anzahl der Zeitpunkte pro Serie (nach Rebinning): {rebinned_data_scaled.shape[1]}")

# --- 6. Transformation ins Long-Format ---
long_format_data = []
# Generische Start-Unix-Zeit in Nanosekunden (z.B. 1. Januar 2020)
start_unix_ns = pd.Timestamp('2020-01-01 00:00:00 UTC').value

# Zeitabstand in Nanosekunden (entspricht dem Abstand in rebinned_time)
time_step_ns = (rebinned_time[1] - rebinned_time[0]) * 1_000_000_000 

print("\n📈 Konvertiere Daten ins Long-Format...")
start_long_format = time.time()
for i in range(num_samples):
    current_series_times = np.array([start_unix_ns + j * time_step_ns for j in range(rebinned_data_scaled.shape[1])])
    series_data = rebinned_data_scaled[i]

    df_temp = pd.DataFrame({
        'date': current_series_times, 
        'data': series_data,          
        'cols': f'Channel_{i}'        
    })
    long_format_data.append(df_temp)

df_time_series_long = pd.concat(long_format_data, ignore_index=True)
print(f"Konvertierung ins Long-Format beendet in {time.time() - start_long_format:.2f} Sekunden.")

print("\n🎉 Lorenz-Zeitreihen im Long-Format erfolgreich erstellt!")
print("\n📦 Erste 5 Zeilen des Long-Format DataFrames:")
print(df_time_series_long.head())

print("\n📊 DataFrame Info:")
df_time_series_long.info()

print(f"\n📏 Gesamt-DataFrame Shape: {df_time_series_long.shape}")

# Optional: Speichern als CSV
# print("\nSpeichere DataFrame als CSV (dies kann etwas dauern)...")
# start_save = time.time()
# df_time_series_long.to_csv("lorenz_time_series_long_format_10k_points_numba.csv", index=False)
# print(f"Speichern beendet in {time.time() - start_save:.2f} Sekunden.")

🚀 Starte Generierung der Lorenz-Zeitreihen mit Numba-Optimierung...
Generiere 5 Lorenz-Kandidaten (Länge: 1000000 Punkte)...
  Simuliere Kandidat 1/5...
  Simuliere Kandidat 2/5...
  Simuliere Kandidat 3/5...
  Simuliere Kandidat 4/5...
  Simuliere Kandidat 5/5...
  Kandidaten-Simulation beendet in 22.60 Sekunden.
Mische und füge Rauschen hinzu für 10 Samples (Numba-optimiert)...
  Mischen und Rauschen beendet in 1.77 Sekunden.
Gesamte Rohdaten-Generierung beendet in 24.37 Sekunden.

Rebinning der Daten mit Faktor 100 (Numba-optimiert)...


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1mNo implementation of function Function(<function mean at 0x106c02fc0>) found for signature:
 
 >>> mean(array(float64, 4d, C), axis=Literal[int](3))
 
There are 2 candidate implementations:
[1m  - Of which 2 did not match due to:
  Overload in function 'array_mean': File: numba/np/old_arraymath.py: Line 424.
    With argument(s): '(array(float64, 4d, C), axis=int64)':[0m
[1m   Rejected as the implementation raised a specific error:
     TypingError: [1mgot an unexpected keyword argument 'axis'[0m[0m
  raised from /opt/miniconda3/envs/ag/lib/python3.11/site-packages/numba/core/typing/templates.py:791
[0m
[0m[1mDuring: resolving callee type: Function(<function mean at 0x106c02fc0>)[0m
[0m[1mDuring: typing of call at /var/folders/bm/0j4cpchs2qx9r4_5l3k5pd5c0000gn/T/ipykernel_71226/3309363379.py (81)
[0m
[1m
File "../../../var/folders/bm/0j4cpchs2qx9r4_5l3k5pd5c0000gn/T/ipykernel_71226/330

In [1]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
import sklearn.preprocessing
from numba import jit # Importiere Numba für Optimierungen
import time # Für Performance-Messung
from tqdm import tqdm # Importiere tqdm für Fortschrittsbalken

# --- 1. Lorenz System Definition (mit Numba-Optimierung) ---
rho = 45.92
sigma = 16.0
beta = 8.0 / 2.0

@jit(nopython=True)
def lorenz_equation(state, t):
    """Die Lorenz-Gleichungen, optimiert mit Numba."""
    x, y, z = state
    return sigma * (y - x), x * (rho - z) - y, x * y - beta * z

def lorenz(t, init):
    """Löst das Lorenz-System für gegebene Startwerte und Zeitpunkte."""
    # odeint ruft die Numba-optimierte lorenz_equation auf
    return odeint(lorenz_equation, init, t)

# --- 2. Numba-optimierte Helferfunktion für Mischen und Rauschen ---
@jit(nopython=True)
def _mix_and_noise(data_out, candidates_arr, randoms, mixtures, noise):
    """Numba-optimierte Logik zum Mischen von Kandidaten und Hinzufügen von Rauschen."""
    num_samples = data_out.shape[0]
    num_mixtures = len(mixtures)
    
    for idx in range(num_samples):
        current_random = randoms[idx]
        selected_candidate_idx = -1
        
        for mix_idx in range(num_mixtures):
            if mix_idx == 0:
                if current_random >= 0 and current_random < mixtures[mix_idx]:
                    selected_candidate_idx = mix_idx
                    break
            else:
                if current_random >= mixtures[mix_idx-1] and current_random < mixtures[mix_idx]:
                    selected_candidate_idx = mix_idx
                    break
        
        if selected_candidate_idx != -1:
            noise_array = np.random.normal(loc=0.0, scale=noise, size=data_out[idx].shape)
            data_out[idx] = candidates_arr[selected_candidate_idx] + noise_array
    return data_out

# --- 3. Haupt-Daten-Generierungsfunktion (ruft Numba-Teil auf) ---
def generate_data_optimized(t_full, samples, mixtures, inits, noise):
    """Generiert die Lorenz-Zeitreihendaten, indem sie Numba-optimierte Teile nutzt."""
    print(f"Generiere {len(inits)} Lorenz-Kandidaten (Länge: {t_full.shape[0]} Punkte)...")
    candidates_list = []
    start_candidates = time.time()
    # tqdm für die Schleife über die initialen Bedingungen
    for init_idx, init in enumerate(tqdm(inits, desc="Simulating Lorenz Candidates")):
        candidates_list.append(lorenz(t_full, init)[:, 0:1])
    candidates_arr = np.array(candidates_list)
    print(f"  Kandidaten-Simulation beendet in {time.time() - start_candidates:.2f} Sekunden.")

    print(f"Mische und füge Rauschen hinzu für {samples} Samples (Numba-optimiert)...")
    data_out = np.zeros((samples, t_full.shape[0], 1), dtype=np.float64)
    randoms = np.random.uniform(size=samples)

    start_mix_noise = time.time()
    # Keine tqdm hier, da _mix_and_noise ein einzelner schneller Numba-Aufruf ist
    data_out = _mix_and_noise(data_out, candidates_arr, randoms, mixtures, noise)
    print(f"  Mischen und Rauschen beendet in {time.time() - start_mix_noise:.2f} Sekunden.")
    
    return data_out, None, t_full

# --- 4. Numba-optimierte Rebinning-Funktion (FEHLER BEHOBEN) ---
@jit(nopython=True)
def rebin(arr, new_shape):
    """
    Numba-optimierte Funktion zum Rebinning eines 2D-Arrays (arr[series, time]).
    Verwendet explizite Schleifen für bessere Numba-Kompatibilität.
    """
    # arr.shape = (num_series, num_raw_points)
    # new_shape = (num_series, num_rebinned_points)

    num_series, num_raw_points = arr.shape
    _, num_rebinned_points = new_shape

    bin_size = num_raw_points // num_rebinned_points 

    out = np.empty(new_shape, dtype=arr.dtype)

    for i in range(num_series): # Iteriere über jede Zeitreihe
        for j in range(num_rebinned_points): # Iteriere über jeden neuen rebinned Punkt
            # Extrahiere das Segment und berechne dessen Mittelwert
            segment_start = j * bin_size
            segment_end = (j + 1) * bin_size
            out[i, j] = np.mean(arr[i, segment_start:segment_end]) # np.mean auf 1D-Segment
            
    return out

# --- 5. Hauptskript: Parameter und Datengenerierung ---
print("🚀 Starte Generierung der Lorenz-Zeitreihen mit Numba-Optimierung...")

# Anpassung: Sehr lange Zeitachse für 10.000 Einträge pro Kanal
# (10.000 Punkte * Rebinning-Faktor 100 = 1.000.000 Rohpunkte pro Simulation)
t_full = np.arange(0.0, 20000.0, 0.02) # Erzeugt 1.000.000 Rohpunkte
num_samples = 10 # Anzahl der Kanäle

# Initialparameter für Lorenz-Simulationen (unverändert)
inits = [
    [ 1.0, 1.0001         , 1.0 ],
    [ 1.0, 1.000001       , 1.0 ],
    [ 1.0, 1.00000001     , 1.0 ],
    [ 1.0, 1.0000000001   , 1.0 ],
    [ 1.0, 1.000000000001 , 1.0 ],
]

# Mischungsverhältnisse generieren (unverändert)
num_candidates = 5
points = 1000 
distr = np.clip(np.random.normal(loc=0.5, scale=0.2, size=points), a_min=0, a_max=1)
mixtures, edges = np.histogram(distr, bins=num_candidates)
mixtures = mixtures / points
tmp = 0
for i, mixture in enumerate(mixtures):
    tmp += mixture
    mixtures[i] = tmp
mixtures_arr = np.array(mixtures, dtype=np.float64) # Für Numba als NumPy-Array

# Daten generieren mit der optimierten Funktion
start_total_generation = time.time()
data_raw, _, t_generated = generate_data_optimized(t_full, samples=num_samples, mixtures=mixtures_arr, inits=inits, noise=7.2)
print(f"Gesamte Rohdaten-Generierung beendet in {time.time() - start_total_generation:.2f} Sekunden.")

# Rebinning-Faktor anpassen für 10.000 Punkte pro Kanal
REBIN_FACTOR = 100 
print(f"\nRebinning der Daten mit Faktor {REBIN_FACTOR} (Numba-optimiert)...")
start_rebin = time.time()
# `data_raw[:, :, 0]` macht es zu einem 2D-Array, wie es `rebin` jetzt erwartet
rebinned_data = rebin(data_raw[:, :, 0], (data_raw.shape[0], data_raw.shape[1] // REBIN_FACTOR)) 
rebinned_time = t_full[::REBIN_FACTOR] # Die Zeitpunkte nach dem Rebinning
print(f"Rebinning beendet in {time.time() - start_rebin:.2f} Sekunden.")

# Skalierung der Daten
print("Skaliere die Daten...")
scaler = sklearn.preprocessing.MinMaxScaler(feature_range=(-10, 10))
scaler.fit(rebinned_data)
rebinned_data_scaled = scaler.transform(rebinned_data)

print(f"Shape der skalierten, rebinned Daten: {rebinned_data_scaled.shape} (Soll: {num_samples}x{t_full.shape[0] // REBIN_FACTOR})")
print(f"Anzahl der Zeitpunkte pro Serie (nach Rebinning): {rebinned_data_scaled.shape[1]}")

# --- 6. Transformation ins Long-Format ---
long_format_data = []
# Generische Start-Unix-Zeit in Nanosekunden (z.B. 1. Januar 2020)
start_unix_ns = pd.Timestamp('2020-01-01 00:00:00 UTC').value

# Zeitabstand in Nanosekunden (entspricht dem Abstand in rebinned_time)
time_step_ns = (rebinned_time[1] - rebinned_time[0]) * 1_000_000_000 

print("\n📈 Konvertiere Daten ins Long-Format...")
start_long_format = time.time()
# tqdm für die Schleife über die Samples beim Konvertieren
for i in tqdm(range(num_samples), desc="Converting to Long Format"): 
    current_series_times = np.array([start_unix_ns + j * time_step_ns for j in range(rebinned_data_scaled.shape[1])])
    series_data = rebinned_data_scaled[i]

    df_temp = pd.DataFrame({
        'date': current_series_times, 
        'data': series_data,          
        'cols': f'Channel_{i}'        
    })
    long_format_data.append(df_temp)

df_time_series_long = pd.concat(long_format_data, ignore_index=True)
print(f"Konvertierung ins Long-Format beendet in {time.time() - start_long_format:.2f} Sekunden.")

print("\n🎉 Lorenz-Zeitreihen im Long-Format erfolgreich erstellt!")
print("\n📦 Erste 5 Zeilen des Long-Format DataFrames:")
print(df_time_series_long.head())

print("\n📊 DataFrame Info:")
df_time_series_long.info()

print(f"\n📏 Gesamt-DataFrame Shape: {df_time_series_long.shape}")

# Optional: Speichern als CSV
# print("\nSpeichere DataFrame als CSV (dies kann etwas dauern)...")
# start_save = time.time()
# df_time_series_long.to_csv("lorenz_time_series_long_format_10k_points_numba_fixed.csv", index=False)
# print(f"Speichern beendet in {time.time() - start_save:.2f} Sekunden.")

🚀 Starte Generierung der Lorenz-Zeitreihen mit Numba-Optimierung...
Generiere 5 Lorenz-Kandidaten (Länge: 1000000 Punkte)...


Simulating Lorenz Candidates: 100%|██████████| 5/5 [00:25<00:00,  5.04s/it]


  Kandidaten-Simulation beendet in 25.23 Sekunden.
Mische und füge Rauschen hinzu für 10 Samples (Numba-optimiert)...
  Mischen und Rauschen beendet in 1.80 Sekunden.
Gesamte Rohdaten-Generierung beendet in 27.04 Sekunden.

Rebinning der Daten mit Faktor 100 (Numba-optimiert)...
Rebinning beendet in 0.13 Sekunden.
Skaliere die Daten...
Shape der skalierten, rebinned Daten: (10, 10000) (Soll: 10x10000)
Anzahl der Zeitpunkte pro Serie (nach Rebinning): 10000

📈 Konvertiere Daten ins Long-Format...


Converting to Long Format: 100%|██████████| 10/10 [00:00<00:00, 580.68it/s]

Konvertierung ins Long-Format beendet in 0.02 Sekunden.

🎉 Lorenz-Zeitreihen im Long-Format erfolgreich erstellt!

📦 Erste 5 Zeilen des Long-Format DataFrames:
           date      data       cols
0  1.577837e+18 -3.502282  Channel_0
1  1.577837e+18 -8.361674  Channel_0
2  1.577837e+18 -4.336743  Channel_0
3  1.577837e+18 -8.960544  Channel_0
4  1.577837e+18  1.055208  Channel_0

📊 DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   date    100000 non-null  float64
 1   data    100000 non-null  float64
 2   cols    100000 non-null  object 
dtypes: float64(2), object(1)
memory usage: 2.3+ MB

📏 Gesamt-DataFrame Shape: (100000, 3)


In [3]:
df_time_series_long.to_csv("lorenz.csv", index=False)